# 2022 League Two EDA

In [51]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

In [52]:
team_stats = pd.read_csv('../footy_data/league_two_teams_2022.csv')
games = pd.read_csv('../footy_data/league_two_games_2022.csv')

In [53]:
games = games[0:69]
games
#Played matches

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Exeter City,Bradford City,Samuel Allison,1,0.0,0.0,...,2.55,3.10,2.85,1.43,2.15,3.65,7.00,1.95,1.83,St James Park (Exeter- Devon)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Forest Green Rovers,Sutton United,James Bell,1,0.0,0.0,...,1.83,3.50,4.20,1.35,1.95,3.30,6.50,1.91,1.87,The Fully Charged New Lawn (Nailsworth- Glouce...
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Harrogate Town,Rochdale,Anthony Coggins,1,0.0,0.0,...,2.20,3.25,3.25,1.37,2.05,3.35,6.50,1.87,1.91,EnviroVent Stadium (Harrogate- North Yorkshire)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Hartlepool United,Crawley Town,Scott Oldham,1,0.0,0.0,...,2.20,3.20,3.30,1.39,2.10,3.45,6.75,1.91,1.87,Victoria Park (Hartlepool- County Durham)
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Mansfield Town,Bristol Rovers,Joshua Smith,1,0.0,0.0,...,2.35,3.15,3.05,1.43,2.15,3.60,7.00,1.95,1.83,One Call Stadium (Mansfield- Nottinghamshire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Northampton Town,Scunthorpe United,Tom Reeves,6,1.5,1.0,...,1.95,3.20,4.25,1.50,2.35,4.25,8.75,2.15,1.67,Sixfields Stadium (Northampton- East Midlands)
65,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Oldham Athletic,Barrow,Carl Boyeson,6,0.0,0.0,...,2.60,3.20,2.70,1.38,2.05,3.40,6.50,1.87,1.91,Boundary Park (Oldham- Greater Manchester)
66,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Port Vale,Rochdale,Marc Edwards,6,1.0,1.5,...,2.05,3.40,3.40,1.39,2.10,3.55,7.00,1.91,1.87,Vale Park (Stoke-on-Trent- Staffordshire)
67,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Stevenage,Swindon Town,John Brooks,6,2.0,3.0,...,2.35,3.05,3.15,1.47,2.25,3.90,7.75,2.00,1.77,The Lamex Stadium (Stevenage- Hertfordshire)


In [54]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]
xg_view[0:50]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
0,Exeter City,Bradford City,1.63,1.68,0,0
1,Forest Green Rovers,Sutton United,1.46,1.17,2,1
2,Harrogate Town,Rochdale,2.17,1.66,3,2
3,Hartlepool United,Crawley Town,1.36,0.80,1,0
4,Mansfield Town,Bristol Rovers,1.86,0.66,2,1
5,Northampton Town,Port Vale,1.12,1.22,1,0
6,Oldham Athletic,Newport County,0.70,1.02,0,1
7,Salford City,Leyton Orient,1.86,1.20,1,1
8,Scunthorpe United,Swindon Town,1.33,1.81,1,3
9,Stevenage,Barrow,1.27,1.40,1,0


# Goal Analysis

In [55]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100

In [56]:
print(f'There were {season_goal_total} goals scored this season')

There were 152 goals scored this season


In [57]:
print(f'There were {season_1h_goal_total} goals scored in the 1H this season, ')

There were 77 goals scored in the 1H this season, 


In [58]:
round(season_1h_goal_total/season_goal_total, 2)
#% of first half goals

0.51

In [59]:
games['total_goals_at_half_time'].count()

69

In [60]:
games['total_goals_at_half_time'].value_counts()

0    29
2    16
1    15
3     6
4     3
Name: total_goals_at_half_time, dtype: int64

# Adding more columns

In [61]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go

Let's write a function to help fill xg in our games df

In [62]:
xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]


In [63]:
xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

In [64]:
def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

In [65]:
fill_stats('Salford City')
fill_stats('Harrogate Town')
fill_stats('Barrow')
fill_stats('Sutton United')
fill_stats('Forest Green Rovers')
fill_stats('Exeter City')
fill_stats('Mansfield Town')
fill_stats('Newport County')
fill_stats('Hartlepool United')
fill_stats('Tranmere Rovers')
fill_stats('Stevenage')
fill_stats('Carlisle United')
fill_stats('Leyton Orient')
fill_stats('Crawley Town')
fill_stats('Colchester United')
fill_stats('Bradford City')
fill_stats('Port Vale')
fill_stats('Oldham Athletic')
fill_stats('Bristol Rovers')
fill_stats('Northampton Town')
fill_stats('Scunthorpe United')
fill_stats('Walsall')
fill_stats('Rochdale')
fill_stats('Swindon Town')

#Reading in stats

In [66]:
games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,home_xg,away_xg,home_xg_against,away_xg_against,home_total_corners,away_total_corners
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Exeter City,Bradford City,Samuel Allison,1,0.0,0.0,...,7.00,1.95,1.83,St James Park (Exeter- Devon),1.09,1.38,1.28,1.21,4.33,6.83
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Forest Green Rovers,Sutton United,James Bell,1,0.0,0.0,...,6.50,1.91,1.87,The Fully Charged New Lawn (Nailsworth- Glouce...,1.23,0.73,0.88,0.94,5.83,4.25
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Harrogate Town,Rochdale,Anthony Coggins,1,0.0,0.0,...,6.50,1.87,1.91,EnviroVent Stadium (Harrogate- North Yorkshire),1.49,1.22,0.99,1.22,7.2,3.83
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Hartlepool United,Crawley Town,Scott Oldham,1,0.0,0.0,...,6.75,1.91,1.87,Victoria Park (Hartlepool- County Durham),1.05,0.99,0.56,1.18,3.2,3.8
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Mansfield Town,Bristol Rovers,Joshua Smith,1,0.0,0.0,...,7.00,1.95,1.83,One Call Stadium (Mansfield- Nottinghamshire),1.49,0.86,1.01,1.27,5.67,4.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Northampton Town,Scunthorpe United,Tom Reeves,6,1.5,1.0,...,8.75,2.15,1.67,Sixfields Stadium (Northampton- East Midlands),1.07,0.66,0.84,1.3,5,5.17
65,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Oldham Athletic,Barrow,Carl Boyeson,6,0.0,0.0,...,6.50,1.87,1.91,Boundary Park (Oldham- Greater Manchester),0.86,0.86,1.08,1.04,5.33,4.5
66,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Port Vale,Rochdale,Marc Edwards,6,1.0,1.5,...,7.00,1.91,1.87,Vale Park (Stoke-on-Trent- Staffordshire),1.07,1.22,1.02,1.22,5.33,3.83
67,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,Stevenage,Swindon Town,John Brooks,6,2.0,3.0,...,7.75,2.00,1.77,The Lamex Stadium (Stevenage- Hertfordshire),0.86,1.16,1.15,1.15,3.83,3.17


# Some more EDA

Let's write a function to help fill xg in our games df

In [ ]:
games['home_xg'] = ""
games['away_xg'] = ""
games['home_xg_against'] = ""
games['away_xg_against'] = ""
games['home_total_corners'] = ""
games['away_total_corners'] = ""

# Creating new columns where our data will go



xg_stats = team_stats[['common_name', 'xg_for_avg_overall']]
corner_stats = team_stats[['common_name', 'corners_per_match']]
xga_stats = team_stats[['common_name', 'xg_against_avg_overall']]

xga_stats.head()

xg_stats = dict(xg_stats.values)
corner_stats = dict(corner_stats.values)
xga_stats = dict(xga_stats.values)

#By making this into dictionaries, it will be easier to call them/update them with the function

def fill_stats(team):
    games['home_xg'] = np.where(games['home_team_name'] == team, xg_stats[team], games['home_xg'])
    games['away_xg'] = np.where(games['away_team_name'] == team, xg_stats[team], games['away_xg'])
    games['home_xg_against'] = np.where(games['home_team_name'] == team, xga_stats[team], games['home_xg_against'])
    games['away_xg_against'] = np.where(games['away_team_name'] == team, xga_stats[team], games['away_xg_against'])
    games['home_total_corners'] = np.where(games['home_team_name'] == team, corner_stats[team], games['home_total_corners'])
    games['away_total_corners'] = np.where(games['away_team_name'] == team, corner_stats[team], games['away_total_corners'])
    
#Function to put team stats into games dataframe

fill_stats('Fulham')
fill_stats('Blackburn Rovers')
fill_stats('West Bromwich Albion')
fill_stats('Queens Park Rangers')
fill_stats('AFC Bournemouth')
fill_stats('Middlesbrough')
fill_stats('Barnsley')
fill_stats('Millwall')
fill_stats('Sheffield United')
fill_stats('Reading')
fill_stats('Cardiff City')
fill_stats('Nottingham Forest')
fill_stats('Hull City')
fill_stats('Blackpool')
fill_stats('Luton Town')
fill_stats('Stoke City')
fill_stats('Swansea City')
fill_stats('Derby County')
fill_stats('Huddersfield Town')
fill_stats('Preston North End')
fill_stats('Coventry City')
fill_stats('Peterborough United')
fill_stats('Birmingham City')
fill_stats('Bristol City')

#Reading in stats

# Some more EDA

# Over/Under 2.5?

In [ ]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5

In [ ]:
games['over/under'].value_counts()

# xG Over/Under 2.5? 

In [ ]:
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under_1.99'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 1.99, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [ ]:
games['xg_over/under'].value_counts()

In [ ]:
games['xg_over/under_1.99'].value_counts()

# First Week

In [ ]:
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [ ]:
first_matchweek = games[0:12]

In [ ]:
first_matchweek['over/under'].value_counts()

In [ ]:
first_matchweek['over/under_1.5'].value_counts()

In [ ]:
first_matchweek['xg_over/under'].value_counts()

In [ ]:
first_matchweek['xg_over/under_1.99'].value_counts()

In [ ]:
games['home_team_goal_count_half_time'].sum()

In [ ]:
games['away_team_goal_count_half_time'].sum()

# Let's take out the final week now and check percentage

In [ ]:
first_41 = games[0:540]
first_41['over/under'].value_counts()

In [ ]:
217/540
# Percentage of games that went over

# When were these goals scored?

Second half stoppage time goals will start at 91 i.e. 90+1 will be 91
First half stoppage time goals will start at 100 i.e. 45+1 = 110 ----

Change stoppage time goals so we can cast them as floats

In [ ]:
goal_timings = goal_timings.replace(to_replace="90'1", value = "91")
goal_timings = goal_timings.replace(to_replace="90'2", value = "92")
goal_timings = goal_timings.replace(to_replace="90'3", value = "93")
goal_timings = goal_timings.replace(to_replace="90'4", value = "94")
goal_timings = goal_timings.replace(to_replace="90'5", value = "95")
goal_timings = goal_timings.replace(to_replace="90'6", value = "96")
goal_timings = goal_timings.replace(to_replace="90'7", value = "97")
goal_timings = goal_timings.replace(to_replace="90'8", value = "98")
goal_timings = goal_timings.replace(to_replace="90'9", value = "99")
goal_timings = goal_timings.replace(to_replace="90'10", value = "100")
goal_timings = goal_timings.replace(to_replace="90'11", value = "101")
goal_timings = goal_timings.replace(to_replace="90'12", value = "102")
goal_timings = goal_timings.replace(to_replace="45'1", value = "110")
goal_timings = goal_timings.replace(to_replace="45'2", value = "111")
goal_timings = goal_timings.replace(to_replace="45'3", value = "112")
goal_timings = goal_timings.replace(to_replace="45'4", value = "113")

In [ ]:
goal_timings['All_goal_timings'] = goal_timings['All_goal_timings'].astype('int')

In [ ]:
timing_totals = goal_timings['All_goal_timings'].value_counts()
timing_totals = pd.DataFrame(timing_totals)

In [ ]:
timing_totals = timing_totals.rename_axis("minute", axis="columns")
timing_totals = timing_totals.rename(columns={"All_goal_timings":"count"})

In [ ]:
timing_totals['section'] = ""
timing_totals.head()

# In how many games did a team have more xG but lose?